# Training of GMM with preprocessed SWaT Dataset
Visual evaluation of the preciseness of discretization and meaningful categories compared to original states. <br>
Wihtin the first subplot we plot a selection of measurement values from the dataset. <br>
The second subplot includes the learned discretizations from the GMM. <br>
The third subplot plots the loglikelihood of the GMM according to the the input data. <br>

To make a statement about the change in states and likelihood, we plot one codeblock with nominal system behavior and the following with anomalous system behavior. 

In [ ]:
cd ..

In [ ]:
import pandas as pd 
from sklearn.mixture import GaussianMixture
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from utils import standardize_data

In [ ]:
def plot_gmm(data, labels, like):
    l = data.shape[1]
    df_sc = standardize_data(data, 'scaler_gmm_SWaT.pkl')
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True)
    for i in range(0, l):
        fig.add_trace(go.Scatter(x=df_sc.index, y=df_sc[df_sc.columns[i]], mode='markers', name=df_sc.columns[i]), 
                                row=1, col=1)
    fig.add_trace(go.Scatter(x=df_sc.index, y=labels, mode='markers', name='categories'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df_sc.index, y=like, mode='markers', name='likelihood'), row=3, col=1)
    fig.show()

In [ ]:
path_norm = 'preprocessed_data/SWaT/swat_norm_p1.csv'
path_anom = 'preprocessed_data/SWaT/swat_anom_p1.csv'

df_norm = pd.read_csv(path_norm)
# rename column due to typo in column name of anom dataset
df_anom = pd.read_csv(path_anom).rename(columns={' MV101':'MV101'})


In [ ]:
# Fitting GMM in nominal data
gmm = GaussianMixture(n_components=4, random_state=0, covariance_type='diag').fit(df_norm)

In [ ]:
# predict labels/discretizations and its likelihood
labels = gmm.predict(df_norm)
labels_anom = gmm.predict(df_anom)
like_gmm = pd.DataFrame(gmm.score_samples(df_norm))
like_gmm_anom = pd.DataFrame(gmm.score_samples(df_anom))


In [ ]:
# plot of nominal data
plot_gmm(data=df_norm[0:13000], labels=labels[0:13000], like=like_gmm[0][0:13000])

In [ ]:
# plot of anomalous data as can be seen between idx 7.000 and 10.000
plot_gmm(data=df_anom[0:30000], labels=labels_anom[0:30000], like=like_gmm_anom[0][0:30000])